<div style="font-size:18pt;padding-top:20px; text-align:center">Домашнее задание 1. <b>Spark RDD API
</b> </div><hr>
<div style="text-align:right;">Куценко А. А <span style="font-style: italic;font-weight: bold;">(ftruf357ft@gmail.com)</span></div>

In [1]:
# Вариант
eu_position = 8
eu_position % 3

2

# Задание 2
Набор данных: [описание](http://jmcauley.ucsd.edu/data/amazon/links.html), [отзывы](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz), [товары](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz)

⚠️ Замечание: для парсинга товаров используйте функцию `eval`.

1. Рассчитайте средний рейтинг товаров из набора данных;
2. Сопоставьте полученные данные из предыдущего пункта с наименованием товаров;
3. Сформируйте RDD товаров с рейтингом меньшим 3. Выведите топ-10 товаров с наименьшим рейтингом;
4. Сохраните результат в постоянное хранилище.

In [2]:
%%bash

# Messages from utils in stderr is not an errros
exec 2>&1

dataset_path="./dataset"

review_file="reviews_Electronics_5.json"
review_link="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz"

meta_file="meta_Electronics.json"
meta_link="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz"

echo "Let's check the availability of the dataset locally"

if [ ! -f "${dataset_path}/${review_file}" ] || [ ! -f "${dataset_path}/${meta_file}" ]; then
    echo "Datset files are missing. Downloading..."
    mkdir -p "$dataset_path"
else
    echo "Dataset files exist. I'm skipping the download"
fi

if [ ! -f "${dataset_path}/${review_file}" ]; then
    echo "Downloading reviews"
    pushd "$dataset_path"
    curl -s "${review_link}" -o "${review_file}.gz"
    gunzip "${review_file}.gz"
    popd
fi

if [ ! -f "${dataset_path}/${meta_file}" ]; then
    echo "Downloading meta"
    pushd "$dataset_path"
    curl -s "${meta_link}" -o "${meta_file}.gz"
    gunzip "${meta_file}.gz"
    popd
fi

pwd
ls -lah "${dataset_path}"

Let's check the availability of the dataset locally
Datset files are missing. Downloading...
~/shared/BigData/HWs/hw1/dataset ~/shared/BigData/HWs/hw1
~/shared/BigData/HWs/hw1
~/shared/BigData/HWs/hw1/dataset ~/shared/BigData/HWs/hw1
~/shared/BigData/HWs/hw1
/home/ubuntu/shared/BigData/HWs/hw1
total 2.0G
drwxrwxr-x 2 ubuntu ubuntu    0 Dec  1 22:06 .
drwxrwxr-x 2 ubuntu ubuntu    0 Dec  1 22:06 ..
-rw-rw-r-- 1 ubuntu ubuntu 633M Dec  1 22:06 meta_Electronics.json
-rw-rw-r-- 1 ubuntu ubuntu 1.4G Dec  1 22:06 reviews_Electronics_5.json


Environment setup

In [3]:
import os
import sys

os.environ["SPARK_HOME"]="/home/ubuntu/BigData/spark"
os.environ["PYSPARK_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/home/ubuntu/ML/anaconda3/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

In [4]:
REVIEW_FILEPATH = "file:///home/ubuntu/shared/BigData/HWs/hw1/dataset/reviews_Electronics_5.json"
META_FILEPATH = "file:///home/ubuntu/shared/BigData/HWs/hw1/dataset/meta_Electronics.json"

OUTPUT_DIRPATH = "file:///home/ubuntu/shared/BigData/HWs/hw1/output"

Create Spark context

In [5]:
import pyspark

In [6]:
conf = pyspark.SparkConf() \
        .setAppName("Hw1App") \
        .set("spark.executor.memory", "1g") \
        .set("spark.executor.core", "2") \
        .set("spark.driver.memory", "2g") \
        .setMaster("local[2]")

sc = pyspark.SparkContext(conf=conf)
sc

<SparkContext master=local[2] appName=Hw1App>

## 1. Рассчитайте средний рейтинг товаров из набора данных.

In [7]:
review_avg_ratings = sc.textFile(REVIEW_FILEPATH) \
    .map(eval) \
    .map(lambda r: (r['asin'], (r['overall'], 1))) \
    .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1])) \
    .mapValues(lambda x: x[0] / x[1])

In [8]:
review_avg_ratings.take(10)

[('B00000J4GE', 4.25),
 ('B00001OPJV', 3.2),
 ('B00004RDMR', 4.470588235294118),
 ('B00004TEN2', 4.071428571428571),
 ('B00004TY7Q', 4.6),
 ('B00004U4MT', 4.363636363636363),
 ('B00004U8K4', 1.8),
 ('B00004Z5M1', 4.681159420289855),
 ('B00005219A', 4.722222222222222),
 ('B00005LD4T', 4.545454545454546)]

## 2. Сопоставьте полученные данные из предыдущего пункта с наименованием товаров.

In [9]:
titles = sc.textFile(META_FILEPATH) \
    .map(eval) \
    .filter(lambda x: 'title' in x ) \
    .map(lambda x: (x['asin'], x['title']))

In [10]:
titles.take(1)

[('0132793040',
  'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker')]

In [11]:
product_ratings = titles.join(review_avg_ratings)
product_ratings_flat = product_ratings.map(lambda x: (x[0], x[1][0], x[1][1]))

In [12]:
product_ratings_flat.take(5)

[('B00000JBRV',
  'Labtec LVA7330 ClearVoice Head Microphone (Discontinued by Manufacturer)',
  4.4),
 ('B00000JFMK', 'Panasonic DVD-L50 Portable DVD Player', 4.75),
 ('B00000JSGF', 'Rio 500 64 MB USB MP3 Player (Gray)', 3.5833333333333335),
 ('B00000K13I',
  'Brother TZ131 Black on Clear 1/2-Inch Labeling Tape (26.2 Feet)',
  4.533333333333333),
 ('B00003CWFA',
  'SuperFlat Mini Navajo White Easy-to-Hide Speaker Cable 50 Feet pc with 4 prs pins (Discontinued by Manufacturer)',
  4.428571428571429)]

## 3. Сформируйте RDD товаров с рейтингом меньшим 3. Выведите топ-10 товаров с наименьшим рейтингом.

In [13]:
poor_goods_by_rating = product_ratings_flat \
    .filter(lambda x: x[-1] < 3.) \
    .sortBy(lambda x: x[-1], ascending=False)

In [14]:
poor_goods_by_rating.take(10)

[('B001LYX3MQ',
  'Li-ion Battery For Apple A1175 MacBook Pro 15-inch series',
  2.9846153846153847),
 ('B000ES8EG0',
  'Philips High Performance Amplified Indoor Uhf/Vhf/Fm/HDTV Antenna (Discontinued by Manufacturer)',
  2.9833333333333334),
 ('B000026D8E',
  'Seagate ST3500320AS Barracuda 7200.11 500 GB SATA 32 MB Cache Bulk/OEM Hard Drive',
  2.981818181818182),
 ('B000LSZ0Q0',
  'Cisco-Linksys WPSM54G Wireless-G Print Server with Multifunction Printer Support',
  2.980769230769231),
 ('B002T0D81C',
  'Pro X Style Red/Cyan 3D Glasses for Movies and Games on Flat Screens',
  2.9767441860465116),
 ('B00475WJEY', '10 Port USB 2.0 Hub', 2.976),
 ('B0036ZHNJW',
  'Importer520 USB 2.0 Ethernet 10/100 Network LAN RJ45 Adapter',
  2.972972972972973),
 ('B00081GX8O', 'Motorola Hands-Free Headset for RAZR V3', 2.9722222222222223),
 ('B000Z4AQFU',
  'Hde EasyCAP USB 2.0 Audio/Video Capture/Surveillance Dongle (AS-EZ-CAP1)',
  2.9711538461538463),
 ('B008CQVWN8',
  'Powerbeats by Dr. Dre In-Ear

## 4. Сохраните результат в постоянное хранилище.

In [15]:
!rm -rf /home/ubuntu/shared/BigData/HWs/hw1/output
!rmdir /home/ubuntu/shared/BigData/HWs/hw1/output

rmdir: failed to remove '/home/ubuntu/shared/BigData/HWs/hw1/output': No such file or directory


In [16]:
poor_goods_by_rating.saveAsTextFile(OUTPUT_DIRPATH)

In [17]:
sc.textFile(OUTPUT_DIRPATH).take(2)

["('B0000C1HGO', 'X10 XCam2 InstantON Tiny Wireless Color Video Camera', 1.8)",
 "('B0045H6FT4', 'Discovery Expedition Wonderwall Entertainment Projector', 1.8)"]

Stop Spark context

In [18]:
sc.stop()